# Patchy particles potential 

Implempentation of five freely diffusing patchy particles (rigid bodies) with two patches each and pair interactions given by the patchy particle potential defined in the main code. In principle, with the current seeds, it will generate a pentameric ring formation. If it does not, you might have different versions of c++ or python.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticle
from msmrd2.integrators import overdampedLangevin as odLangevin

In [ ]:
# Define arbitrarily large list of patchy particles
np.random.seed(seed=1) # seed 1 good used for pentameric ring formation 
numparticles = 5
boxsize = 3
D = 1.0
Drot = 1.0
pyPartlist = [] 
positionList = [np.array([-1.0,0,0]), np.array([0.,0.,0.]), np.array([1.0,0.,0.]), 
                np.array([2.0,0.,0.]), np.array([3.0,0.,0.])]
orientationList = [ np.array([1.,0.,0.,0.]), np.array([np.cos(np.pi/2),0.,0.,np.sin(np.pi/2)])]

for i in range(numparticles):
    
    overlap = True
    while overlap:
        position = np.array([boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize])
        overlap = False     
        for j in range(len(pyPartlist)):
            if np.linalg.norm(position - pyPartlist[j].position) < 2.0:
                overlap = True
                continue
                
    orientation = np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])
    orientation = orientation/np.linalg.norm(orientation)
    part = msmrd2.particle(D, Drot, position, orientation)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = 2 # seed 2 good used for pentameric ring formation; seed = -1 uses random device as seed
bodytype = 'rigidbody' # three rotational degrees of freedom
integrator = odLangevin(dt, seed, bodytype) 
integrator.setKbT(0.1)

In [ ]:
# Define boundary (choose either spherical or box)
sphereBoundary = msmrd2.sphere(boxsize,'reflective')
integrator.setBoundary(sphereBoundary)

In [ ]:
# Define Patchy Particle potential
sigma = 1.0
strength = 200.0
patch1 = np.array([1.,0.,0.])
patch2 = np.array([np.cos(3*np.pi/5.0),np.sin(3*np.pi/5.0),0.])
patchesCoordinates = [patch1, patch2]
potentialPatchyParticle = patchyParticle(sigma, strength, patchesCoordinates)
integrator.setPairPotential(potentialPatchyParticle)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 5000000
stride = 2500 #1000
datafile  = open('../../data/vmd/patchyParticles.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(3*len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[0], part.orientation)
            v2 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[1], part.orientation)
            datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticles(numparticles = numparticles, 
                                     outfname = "patchyParticles", 
                                     tclfname = "../../data/vmd/patchyParticles_2vmd.tcl")
print("Percentage complete: ", 100, " %")

To load the movie go to /data/vmd and run in a terminal "vmd -e patchyPacrticles_2vmd.tcl".

## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions of first and last particle in list
timesteps = 10
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position (first particle)", "Position (last particle)"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[numparticles-1].position)))
    integrator.integrate(partlist)

# Calculate potential and plot

Plots the potential for the aligned and misaligned case

In [ ]:
# Calculate potential for aligned and misaligned molecules
def patchyParticlesEvaluate(p1,p2,or1,or2):
    part1 = msmrd2.particle(1.0, 1.0, p1, or1)
    part2 = msmrd2.particle(1.0, 1.0, p2, or2)
    return potentialPatchyParticle.evaluate(part1, part2)

dist = np.arange(0, 2*sigma, 0.01)
pos1 = np.array([0., 0., 0.])
pos2 = np.array([[i, 0., 0.] for i in dist])
phi1 = 0.0
phi2 = np.pi
phi1p = np.pi
phi2p = 0.0
u = np.array([0.,0.,1.]) #rotationAxis
# Quaternions that rotate phi along axis u
theta1 = np.array([np.cos(0.5*phi1 ),
                   np.sin(0.5*phi1 )*u[0],
                   np.sin(0.5*phi1 )*u[1],
                   np.sin(0.5*phi1 )*u[2]]) 
theta2 = np.array([np.cos(0.5*phi2 ),
                   np.sin(0.5*phi2 )*u[0],
                   np.sin(0.5*phi2 )*u[1],
                   np.sin(0.5*phi2 )*u[2]]) 
theta1p = np.array([np.cos(0.5*phi1p ),
                   np.sin(0.5*phi1p )*u[0],
                   np.sin(0.5*phi1p )*u[1],
                   np.sin(0.5*phi1p )*u[2]]) 
theta2p = np.array([np.cos(0.5*phi2p ),
                   np.sin(0.5*phi2p )*u[0],
                   np.sin(0.5*phi2p )*u[1],
                   np.sin(0.5*phi2p )*u[2]]) 

potantialAligned = [patchyParticlesEvaluate(pos1,p2,theta1,theta2) for p2 in pos2]
potantialMisaligned = [patchyParticlesEvaluate(pos1,p2,theta1p,theta2p) for p2 in pos2]

In [ ]:
# Plot potential
plt.figure(figsize=(7,5))
fsize = 22
plt.rcParams.update({'font.size': fsize})
plt.plot(dist, potantialAligned, label = r'aligned', lw =2)
plt.plot(dist, potantialMisaligned, label = r'misaligned', lw=2)
plt.plot(dist, 0*dist, '--k', lw=0.5)
#plt.legend(fancybox=True)
plt.legend(fontsize=28, labelspacing=2, framealpha=1.0, edgecolor='white', fancybox=True, bbox_to_anchor=(1.3, 1.0))
plt.xlim([0,2*sigma])
plt.xlabel(r'Relative distance')
plt.ylabel(r'Potential')
plt.yticks([])
#plt.savefig('patchyParticles_potential.pdf', bbox_inches='tight')